# Scrapy Note まとめ

<https://sugiaki1989.gitbook.io/scrapy-note/>

- Reference
  - <https://scrapy-docs-ja.readthedocs.io/ja/latest/index.html>

## インストール

```bash
pip install Scrapy
```

## プロジェクトの作成

```bash
# scrapy startproject <project-name>
scrapy startproject sample_project
```

```bash
# ディレクトリの移動
cd sample_project
```

```bash
# クローラー名（spider名）とスクレイピング対象のURLを指定
# scrapy genspider <spider-name> <page-url>
scrapy genspider sample example.com
```

```bash
# ディレクトリの表示
tree

.
├── scrapy.cfg
└── sample_project
    ├── __init__.py
    ├── items.py
    ├── middlewares.py
    ├── pipelines.py
    ├── settings.py
    └── spiders
        ├── __init__.py
        └── sample.py
```

- Scrapy
  - 収集するデータを定義する
  - 抽出する場所と範囲を決定する
  - 抽出するための方法を記述する
  - 抽出したアイテムを処理する方法を決定する
  - クローラーを起動し、スクレイピングを実行する

- items.py
  - 出力されるデータの形式を定義して、構造化されたデータを格納する
  - Itemクラス
  - Itemオブジェクト

```python
# items.py
class Product(scrapy.Item):
    X = scrapy.Field()
    Y = scrapy.Field()
    Z = scrapy.Field()
```

- spiders/spider.py (sample.py)
  - `scrapy genspider`コマンドで生成したspidersディレクトリ内の実行ファイル。
  - クローラーの動きやスクレイピング方法を定義するクラス。
- QuotesSpider
  - allowed_domainsで許可されたドメイン内において、
  - start_urlsで指定されたURLを起点にQuotesSpiderは動き始める。
  - URLのRequestを生成するstart_requestsメソッドとRequestのコールバック関数としてparseメソッドを呼び出す。
  - コールバック関数は、
    - ResponseであるWebページをセレクターを使用してページ内容をパースして、
    - データを抽出を行い、
    - 辞書形式でItemを返す。
  - Itemは、データベースやCSV、JSONに保存できる。

```python
# spiders/spider.py (sample.py)
import scrapy


class QuotesSpider(scrapy.Spider):
    name = 'quotes'
    allowed_domains = ['quotes.toscrape.com']
    start_urls = ['http://quotes.toscrape.com/']

    def parse(self, response):
        quotes = response.xpath('//*[@class="quote"]')
        for quote in quotes:
            text = quote.xpath('.//*[@class="text"]/text()').get()
            author = quote.xpath('.//*[@class="author"]/text()').get()
            tags = quote.xpath('.//*[@class="tag"]/text()').getall()

            yield {
                "text": text,
                "author": author,
                "tags": tags
            }

        next_page_url = response.xpath('//*[@class="next"]/a/@href').get()
        abs_next_page_url = response.urljoin(next_page_url)
        if next_page_url is not None:
            yield scrapy.Request(abs_next_page_url, callback=self.parse)
```

- name:
  - ターミナルでクローラーを起動させる場合に必要なクローラー名。
- allowed_domains:
  - クロールできるドメイン名。
- start_urls:
  - クロールを開始するURLのリスト。

&nbsp;

```bash
# scrapy crawl <crawler-name> <option> <filename>
scrapy crawl quotes -o output.json
```

- 上記スクリプトが実行されると、scrapyが動く。
- `-o`: アウトプットする形式を選べる。
  - XML, CSV, JSON, JSON-LINES。

- pipelines.py
  - SPIDERによってスクレイピングされたItemは、ITEM-PIPELINESに送られて処理される。
  - データのバリデーション
  - データクリーニング
  - 画像のサイズ変更、その他の画像処理
  - データベースへの保存

```python
# pipeline.py
# 特定の条件に当てはまるものを削除する
from scrapy.exceptions import DropItem

class PricePipeline(object):

    vat_factor = 1.15

    def process_item(self, item, spider):
        if item.get('price'):
            if item.get('price_excludes_vat'):
                item['price'] = item['price'] * self.vat_factor
            return item
        else:
            raise DropItem("Missing price in %s" % item)
```

- ITEM-PIPELINES コンポーネントをアクティブにするために、setting.pyに設定を追加する。

```python
# setting.py
ITEM_PIPELINES = {
    'myproject.pipelines.PricePipeline': 300
}
```

- middlewares.py
  - Scrapyを拡張させるために使用される機能。
  - Downloader Middleware:
    - Webページのダウンロード処理を拡張する。
    - <https://scrapy-docs-ja.readthedocs.io/ja/latest/topics/downloader-middleware.html>
  - Spider Middleware:
    - コールバック関数の処理を拡張する。
    - <https://scrapy-docs-ja.readthedocs.io/ja/latest/topics/spider-middleware.html>

- Scrapy Architecture

![](scrapy_architecture_02.png)

- 1.Scrapyエンジンは、Spidersからクロールする最初のRequestsを取得。
- 2.Scrapyエンジンは、SchedulerでRequestsをスケジュールリングし、クロールする次のRequestsを求める。
- 3.Schedulerは、RequestsをScrapyエンジンに返す。
- 4.Scrapyエンジンは、DownloaderにRequestsを送り、Middlewareを通過させる。
- 5.ページのダウンロードが完了すると、 Downloaderは、そのページのレスポンスを生成し、Scrapyエンジンに送信。 Middlewareを通過させる。
- 6.Scrapyエンジンは、 Downloaderからレスポンスを受け取り、それをSpidersに送って処理する。Middlewareを通過させる。
- 7.Spidersはレスポンスを処理し、スクレイピングされたItemと新しいRequestsをScrapyエンジンに返し、Middlewareを通過させる。
- 8.Scrapyエンジンは処理済みのItemを Item・pipelineに送信し、処理済みのRequestsをSchedulerに送信し、可能なら次のクロールを行う。
- 9.Schedulerからの要求がなくなるまで、プロセスは(ステップ1から)繰り返される。